In [1]:
import numpy as np
import os
from PIL import Image
from netCDF4 import Dataset
from tqdm import tqdm
from utils import *
from model import *
import glob
from shutil import copy

In [2]:
cached_window_2D = window_2D(484, 2)

In [3]:
model = tf.keras.models.load_model("save_cloud_model", custom_objects={"weighted_binary_crossentropy": weighted_binary_crossentropy})

In [4]:
def subdivs_compute(mdgm_path, split_mdgm=668, c=3, bounding=None):
    """
    Computes a cloudmask for a given mdgm.

    Parameters:
    mdgm_path -- path to an mdgm image in a file directory that resembles https://doi.org/10.7910/DVN/U3766S. See getInfo_compute() in utils for description of folder structure
    split_mdgm -- integer equal to the input size of the model (default 668)
    probs -- returns the cloud probablity if True, returns the binary classification if False (default False)
    c -- integer that alters the automated polar extents by c degrees. See getPolarBoundsNew() in utils for usage. Not used if bounding is not None (default 3)
    bounding -- optional tuple of four coordinates to manually dictate the bounding box instead of using the ls value. Tuple should be in format (xmin, xmax, ymin, ymax) (default None)
    """
    # gets output mask size (668 --> 484)
    split_mask = int(check_UNET_num(split_mdgm))

    mdgm = Image.open(mdgm_path)

    if bounding is not None:
        (xlow, xhigh, ylow, yhigh) = bounding
        xhigh, yhigh = xhigh - 1, yhigh - 1
    else:
        lat_n, lat_s = get_polar_bounds_new(get_info_compute(mdgm_path)[0], c)

        (ylow, yhigh) = (bound_to_pixel(lat_n), bound_to_pixel(lat_s))
        (xlow, xhigh) = get_black_bounds(mdgm, "lr")

    # pads mdgm given bounds calculated above
    padded_mdgm = pad_mdgm(mdgm, xhigh, xlow, yhigh, ylow)

    # calculates subdivision data: x_splits, y_splits = number of splits for each axis; xSize, ySize = size of each subdivision
    x_splits = int(np.ceil((xhigh + 1 - xlow) / split_mask * 2 - 1))
    y_splits = int(np.ceil((yhigh + 1 - ylow) / split_mask * 2 - 1))
    xSize = (xhigh + 1 - xlow) / (x_splits + 1)
    ySize = (yhigh + 1 - ylow) / (y_splits + 1)

    # creates cloudmask array with -999 defaults
    full_cloudmask = np.ndarray((x_splits * y_splits, 2, mdgm.height, mdgm.width))
    full_cloudmask.fill(-999)

    for i in range(x_splits):
        for j in range(y_splits):
            xmin = round(xSize * i)
            xmax = xmin + split_mdgm

            ymin = round(ySize * j)
            ymax = ymin + split_mdgm

            # safety: final subdivisions are based on ends of mdgm, not the running split count
            if i == x_splits - 1:
                xmax = padded_mdgm.width
                xmin = xmax - split_mdgm

            if j == y_splits - 1:
                ymax = padded_mdgm.height
                ymin = ymax - split_mdgm

            sub_mdgm = np.expand_dims(
                np.array(padded_mdgm.crop((xmin, ymin, xmax, ymax))), axis=0
            )

            # model prediction for each subdivision
            full_cloudmask[
                i * y_splits + j,
                :1,
                ylow + ymin : ylow + ymin + split_mask,
                xmin + xlow : xmin + xlow + split_mask,
            ] = model.predict(sub_mdgm, verbose=0).reshape((1, 484, 484))

            full_cloudmask[
                i * y_splits + j,
                1:,
                ylow + ymin : ylow + ymin + split_mask,
                xmin + xlow : xmin + xlow + split_mask,
            ] = cached_window_2D

    # stitches predicted subdivisions together by selecting the predictions closest to the centers of their subdivision
    index = np.expand_dims(np.argmax(full_cloudmask[:, 1, ...], axis=0), 0)
    cloudmask = np.take_along_axis(full_cloudmask[:, 0, ...], index, axis=0)[0]

    # all_three_only mask
    nan_img = np.array(mdgm)
    cloudmask[
        ((nan_img[..., 1] < 20) & (nan_img[..., 2] < 20))
        | (nan_img[..., 2] < 30)
        | (nan_img[..., 0] == 0)
        | (nan_img[..., 1] == 0)
        | (nan_img[..., 2] == 0)
    ] = -999
    
    return mdgm, cloudmask

In [ ]:
# runs through all mdgms located at master_path and stores the model's output at save_path

# master_path must follow format of https://doi.org/10.7910/DVN/U3766S. The folder should contain one or more phases of data.
master_path = "data/raw"
save_path = "data/preds"


def make_netCDF(cloudmask_in, save_mask_path):
    """
    Creates NetCDF file for an inputted cloudmask array.
    """
    root = Dataset(save_mask_path, "w", format="NETCDF4_CLASSIC")
    root.set_fill_off()

    # dimensions
    root.createDimension("x", 3600)
    root.createDimension("y", 1801)

    # variables
    longitude = root.createVariable("longitude", "float32", ("x",), zlib=True)
    latitude = root.createVariable("latitude", "float32", ("y",), zlib=True)
    cloudmask = root.createVariable("cloudmask", "float32", ("y", "x"), zlib=True)

    # data
    lon_range = np.linspace(-180, 179.9, 3600, dtype=np.float32)
    lat_range = np.linspace(-90, 90, 1801, dtype=np.float32)
    longitude[:] = lon_range
    latitude[:] = lat_range
    cloudmask[...] = cloudmask_in
    root.close()


try:
    os.mkdir(save_path)
except:
    pass

for phase in [
    i for i in os.listdir(master_path) if os.path.isdir(os.path.join(master_path, i))
]:
    print(phase)
    for subphase in tqdm(
        [
            i
            for i in os.listdir(os.path.join(master_path, phase))
            if os.path.isdir(os.path.join(master_path, phase, i))
        ]
    ):
        os.makedirs(os.path.join(save_path, phase, subphase, "mdgms"))
        os.makedirs(os.path.join(save_path, phase, subphase, "cloudmasks"))
        copy(
            os.path.join(master_path, phase, subphase, "{}_ls.txt".format(subphase)),
            os.path.join(save_path, phase, subphase, "{}_ls.txt".format(subphase)),
        )
        for mdgm_path in glob.glob(os.path.join(master_path, phase, subphase, "*.jpg")):
            print(mdgm_path)
            sub_and_day = mdgm_path.split(os.sep)[-1][:9]
            (mdgm, clouds) = subdivs_compute(mdgm_path)
            copy(
                os.path.join(mdgm_path),
                os.path.join(
                    save_path, phase, subphase, "mdgms", "{}.jpg".format(sub_and_day)
                ),
            )
            make_netCDF(
                np.flip(clouds, axis=0),
                os.path.join(
                    save_path,
                    phase,
                    subphase,
                    "cloudmasks",
                    "cloudmask_{}.ncdf".format(sub_and_day),
                ),
            )